# Trabalho Prático 2 de Estruturas Criptográficas

- **Autores:** (Grupo 9)
     - Nelson Faria (A84727)
     - Miguel Oliveira (A83819)

## *Criptosistemas pós-quânticos PKE/KEM* - NTRU

Implementar o esquema NTRU em classes *Python/SageMath* apresentando, para cada um, as versões **KEM-IND-CPA** e **PKE-IND-CCA**.

Nota: Baseado no documento NTRU (https://ntru.org/f/ntru-20190330.pdf)

Conjuntos de parâmetros ntru-hps específicos são denotados ntruhps(n), por exemplo **ntruhps4096821** é ntru-hps com **n = 821**. O conjunto de parâmetros ntru-hps recomendado é ntruhps4096821, de acordo com o que foi submetido na 3ªronda do PQC.

### NTRU - PKE

Para a implementação do PKE-IND-CCA decidímos seguir a submissão da NTRU especificada no documento, especificação essa de acordo com o respetivo documento apresenta já um PKE-IND-CCA e KEM-IND-CCA, pelo que desta forma só foi necessário implementar as duas versões especificadas na submissão.

Primeiramente, na inicialização da classe, são recebidos os parámetros de acordo com o NTRU-HPS e ainda criados os anéis necessários (Zx, R e Rq), sendo que os outros anéis como o Sq e o S3 não foram necessários, pois usou-se sempre os valores de forma arredondada(para o caso do 3, em vez de ser valores entre {0,1,2}, usa-se valores entre {-1,0,1}).

De seguida daremos então início à explicação da geração do par de chaves pública e privada, da cifragem e da decifragem:

**Geração da chave pública *h* e da chave privada (f,fq,hq) *(função geraChaves())***

  - A seed é uma string de bits aleatórios que serve para gerar os polinómios ternários f e g. Esta seed deve ter tamanho suficiente para posteriormente ser dividida a metade para gerar os polinomios f e g;
  - Os polinomios f e g são gerados desta forma: (f,g) $<-$ Sample_fg(seed), onde o nesta função *Sample_fg()* o objetivo é pegar na seed e gerar um polinómio ternário f e g, sendo estes gerados de forma muito simples:
     - Caso o bit seja igual a 0, então fica 1 como coeficiente, caso seja 1 fica -1 como coeficiente. Depois, e para que o polinómio tenha o tamanho certo, completa-se tudo com 0´s e faz-se o *shuffle* dos elementos da lista resultante;
  - Depois de gerados os polinómios f e g, pode-se então passar para a fase de cálculo dos elementos da chave privada *(f,fp,hq)* e da chave pública *h*. Como para isso são necessários alguns cálculos de inversas, então o que se fez foi fazer as várias inversas, mas tendo em atenção que alguns podem não possuir inversa, pelo que neste caso foi necessário fazer um ciclo para que, caso haja uma falha numa das inversas, então voltar a iterar e a gerar novos polinómios f e g. Assim, os cálculos necessários resumidamente são os seguintes:
    - fp $<-$ (1/f) mod(3,$\Phi(n)$)
    - fq $<-$ (1/f) mod(q,$\Phi(n)$)
    - h $<-$ (3.g.fq) mod(q,$\Phi(1)$$\Phi(n)$)
    - hq $<-$ (1/f) mod(q,$\Phi(n)$)
    

**Cifragem(função *cifra()*)**

  - A função cifra recebe como parâmetros a chave pública $h$, um $r$ e um $m$ que será a mensagem a enviar. O $r$ é um parâmetro gerado de forma aleatória.
  - Depois, é calculado o criptograma através da expressão: c $<-$ (r.h + m) mod(q,$\Phi(n)$) (Normalmente, no lugar do m, deveria de estar o Lift(m), mas de acordo com o NTRU-HPS, o Lift(m) = m);


**Decifragem (função *decifra()*)**

  - A função decifra recebe como parâmetros a chave privada $(f,fp,hq)$ e o criptograma $c$;
  - a $<-$ (c.f) mod(q,$\Phi(1)$$\Phi(n)$);
  - m $<-$ (a.fp) mod(3,$\Phi(n)$);
  - r $<-$ ((c-m').hq) mod(q,$\Phi(n)$), onde m' é igual a Lift(m) = m, de acordo com o NTRU-HPS;
  - Se os polinomios (r,m) nao forem ternarios, retorna (0,0,1), senão retorna (r,m,0).

In [8]:
import random, hashlib
import numpy as np

# Baseado no esquema da página 25 do documento https://ntru.org/f/ntru-20190330.pdf
# https://latticehacks.cr.yp.to/ntru.html

class NTRU_PKE(object):
    
    def __init__(self, N=821, Q=4096, D=495, timeout=None):
        
        # Todas as inicializações de parâmetros são baseadas na submissao com os parâmetros do ntruhps4096821, onde n = 821
        self.n = N
        self.q = Q
        self.d = D
        
        # Definição dos aneis
        Zx.<x>  = ZZ[]
        self.Zx = Zx
        Qq = PolynomialRing(GF(self.q), 'x')
        x = Zx.gen()
        y = Qq.gen()
        R = Zx.quotient(x^self.n-1)
        self.R = R
        Rq = QuotientRing(Qq, y^self.n-1)
        self.Rq = Rq
        
          
    # Gera uma string de bits com tamanho size e d 1's
    def randomBitString(self, size):
        
        # Gera uma sequencia de n bits aleatorios
        u = [random.choice([0,1]) for i in range(size)]
        # Mistura os valores da lista, so para aumentar a aleatoriedade
        random.shuffle(u)
        return u
    
    
    # Verifica se um polinomio e ternario
    def isTernary(self, f):
        
        res = True
        v = list(f)
        for i in v:
            if i > 1 or i < -1:
                res = False
                break
        return res
    
    
    # Produz o polinomio f modulo q. Mas, em vez de ser entre 0 e q-1, fica entre -q/2 e q/2-1
    def arredonda_mod(self, f, q):
        
        g = list(((f[i] + q//2) % q) - q//2 for i in range(self.n))
        return self.Zx(g)
    
    
    # Produz a inversa de um polinomio f modulo x^n-1 modulo p, em que p é um numero primo. 
    def inversa_modP(self, f, p):
        
        T = self.Zx.change_ring(Integers(p)).quotient(x^self.n-1)
        return self.Zx(lift(1 / T(f)))
    
    
    # Como a funcao de cima, mas o q aqui é uma potencia de 2
    def inversa_mod2(self, f, q):
        
        assert q.is_power_of(2)
        g = self.inversa_modP(f, 2)
        while True:
            r = self.arredonda_mod(self.R(g*f), q)
            if r == 1:
                return g
            g = self.arredonda_mod(self.R(g*(2 - r)), q)
    
    
    # Gera um polinomio ternario
    def Ternary(self, bit_string):
        
        # cria um array
        result = []
        # Itera d vezes
        for j in range(self.d):
            # Se o bit for 0, acrescenta 1, senao -1
            if bit_string[j] == 0:
                result += [1]
            elif bit_string[j] == 1:
                result += [-1]
        # Preenche com 0's o array restante
        result += [0]*(self.n-self.d)
        # Mistura os valores do array
        random.shuffle(result)

        return self.Zx(result)
    
    
    # Gera um polinomio f em Lf (no nosso caso, em T+) e um polinomio g em Lg (no nosso caso, em {φ1.v : v € T+})
    def Sample_fg(self, seed):
        
        x = self.R.gen()
        
        # Parse de fg_bits em f_bits||g_bits
        f_bits = seed[:self.d]
        g_bits = seed[self.d:]
        
        # Definir f = Ternary_Plus(f_bits)
        f = self.Ternary(f_bits)
        # Definir g0 = Ternary_Plus(g_bits)
        g = self.Ternary(g_bits)
        
        return (f,g)
    
    
    # Gera um polinomio r em Lr (no nosso caso, em T) e um polinomio m em Lm (no nosso caso, em T)
    def Sample_rm(self, coins):
        
        # sample_iid_bits = 8*n - 8
        sample_iid_bits = 8*self.n - 8
        
        # Parse de rm_bits em r_bits||m_bits
        r_bits = coins[:sample_iid_bits]
        m_bits = coins[sample_iid_bits:]
        
        # Set r = Ternary(r_bits)
        r = self.Ternary(r_bits)
        # Set m = Ternary(m_bits)
        m = self.Ternary(m_bits)
        
        return (r,m)
    
    
    # Funcao usada para gerar o par de chaves pública e privada
    def geraChaves(self, seed):
        
        while True:
            try:
                (f,g) = self.Sample_fg(seed)
                # fp <- (1/f) mod(3;φn)
                fp = self.inversa_modP(f, 3)
                # fq <- (1/f) mod (q;φn)
                fq = self.inversa_mod2(f, self.q)
                # gq <- (1/f) mod (q;φn) (so para garantir que h e invertivel)
                gq = self.inversa_mod2(g, self.q)
                # h <- (3.g.fq) mod(q;φ1φn)
                h = self.arredonda_mod(3*self.R(g*fq), self.q)
                # hq <- (1/h) mod (q;φn)
                hq = self.inversa_mod2(h, self.q)
                break
            except:
                # Para que a nova iteracao tenha uma nova seed
                seed = self.randomBitString(2*self.d)
                pass
        
        return {'sk' : (f,fp,hq) , 'pk' : h}

    
    # Recebe como parametros a chave publica e o tuplo (r,m)
    def cifra(self, h, rm):
    
        r = rm[0]
        m = rm[1]
        # c <- (r.h + m') mod (q,φ1φn)
        c = self.arredonda_mod(self.R(h*r) + m, self.q)
        
        return c
    
    
    # Recebe como parametros a chave privada (f,fp,hq) e ainda o criptograma
    def decifra(self, sk, c):
        
        # a <- (c.f) mod(q,φ1φn)
        a = self.arredonda_mod(self.R(c*sk[0]), self.q)
        # m <- (a.fp) mod(3,φn)
        m = self.arredonda_mod(self.R(a * sk[1]), 3)
        # r <- ((c-m').hq) mod(q,φn)
        aux = (c-m) * sk[2]
        r = self.arredonda_mod(self.R(aux), self.q)
        # Se os polinomios nao forem ternarios, retorna erro
        if not self.isTernary(r) and not self.isTernary(m):
            (0,0,1)
        return (r,m,0)
        

#### Testagem da classe definida acima:

In [9]:
# Parametros do NTRU (ntruhps4096821)
N=821
Q=4096
D=495

# Inicializacao da classe
ntru = NTRU_PKE(N,Q,D)

print("[Teste da cifragem e decifragem]")
keys = ntru.geraChaves(ntru.randomBitString(2*D))
rm = ntru.Sample_rm(ntru.randomBitString(11200))

c = ntru.cifra(keys['pk'], rm)

rmDec = ntru.decifra(keys['sk'], c)

if rmDec[0] == rm[0] and rmDec[1] == rm[1] and rmDec[2] == 0:
    print("As mensagens e os r's são iguais!!!!")
else:
    print("A decifragem falhou!!!!")

[Teste da cifragem e decifragem]
As mensagens e os r's são iguais!!!!


### NTRU - KEM

De referir novamente que para a implementação do PKE-IND-CCA decidímos seguir a submissão da NTRU especificada no documento, especificação essa de acordo com o respetivo documento apresenta já um PKE-IND-CCA e KEM-IND-CCA, pelo que desta forma só foi necessário implementar as duas versões especificadas na submissão.

Ao iniciar a classe KEM, esta aproveita a classe **NTRU_PKE** definida anteriormente para recorrer principalmente às funções *geraChaves()*, *cifra()* e *decifra()* lá definidas, sendo ainda definidos os parâmetros de segurança de acordo com o NTRU-HPS que também são passados à classe PKE.

**Geração da chave pública *h* e da chave privada (f,fq,hq,s) *(função geraChaves())***

  - Para a geração do par de chaves pública e privada, usou-se a função de geração de chaves definida na classe **NTRU_PKE**, sendo que deste modo já se consegue ter os parâmetros (f,fq,hq) e ainda o h;
  - Deste modo, só fica a faltar a geração do parâmetro $s$, que é feita recorrendo a s $<-^\$$ {0,1}$^{256}$, ou seja, à geração de uma sequência de bits aleatórios.

**Encapsulamento e geração da chave *(função encapsula())***

  - A função encapsula recebe como parâmetro a chave pública e produz o par (c,k), onde c é o 'encapsulamento' da chave e o k é a chave em si;
  - Depois é gerada uma sequência aleatória de bits: coins $<-^\$$ {0,1}$^{256}$;
  - É gerado um polinómio ternário $r$ e $m$ de forma aleatória recorrendo à função: (r,m) $<-$ Sample_rm(coins);
  - De seguida, procede-se à cifragem do (r,m) através da função *cifra()*: c $<-$ Encrypt(h,(r,m)), sendo este c o 'encapsulamento' da chave;
  - Por fim, faz-se o hash de r e m para obter a chave simétrica: k $<-$ H1(r,m).
  

**Desencapsulamento da chave *(função desencapsula())***

  - A função desencapsula recebe como parâmetros a chave secreta/privada e o encapsulamento da chave e retorna a chave simétrica k;
  - Primeiro, é feito logo a decifragem do encapsulamento da chave c através da função *decifra()* definida no PKE anterior: (r,m,fail) $<-$ Decrypt((f,fp,hq),c), lembrando que a chave secreta que esta função recebe não incluí o parâmetro $s$;
  - Faz-se o hash de r e m para obter a chave simétrica: k1 $<-$ H1(r,m);
  - Faz-se o hash de s e c para obter uma outra chave diferente para o caso de o desencapsulamento falhar: k2 $<-$ H2(s,c);
  - Se fail = 0, então retorna k1, senão retorna k2.

In [10]:
import random, hashlib
import numpy as np

# Baseado no esquema da página 25 do documento https://ntru.org/f/ntru-20190330.pdf
# https://latticehacks.cr.yp.to/ntru.html

class NTRU_KEM(object):
    
    def __init__(self, N=821, Q=4096, D=495, timeout=None):
        
        # Todas as inicializações de parâmetros são baseadas na submissao com os parâmetros do ntruhps4096821, onde n = 821
        self.n = N
        self.q = Q
        self.d = D
        
        # inicializacao da instancia NTRU_PKE
        self.pke = NTRU_PKE(self.n, self.q, self.d)
    
    
    #função para calcular o hash (recebe dois polinomios)
    def Hash1(self, e0, e1):
        
        ee0 = reduce(lambda x,y: x + y.binary(), e0.list() , "")
        ee1 = reduce(lambda x,y: x + y.binary(), e1.list() , "")
        m = hashlib.sha3_256()
        m.update(ee0.encode())
        m.update(ee1.encode())
        return m.hexdigest()
    
    
    #função para calcular o hash (recebe uma string de bits e um polinomio)
    def Hash2(self, e0, e1):
        
        ee1 = reduce(lambda x,y: x + y.binary(), e1.list() , "")
        m = hashlib.sha3_256()
        m.update(e0.encode())
        m.update(ee1.encode())
        return m.hexdigest()
    
    
    # Funcao usada para gerar o par de chaves pública e privada(acrescenta ao geraChaves1() um s)
    def geraChaves(self, seed):
        
        # ((f,fp),h) <- KeyGen'()
        keys = self.pke.geraChaves(seed)
        # s <-$ {0,1}^256
        s = ''.join([str(i) for i in self.pke.randomBitString(256)])
        
        # return ((f,fp,hq,s),h)
        return {'sk' : (keys['sk'][0],keys['sk'][1],keys['sk'][2],s) , 'pk' : keys['pk']}
        
    
    # Funcao que serve para encapsular a chave que for acordada a partir de uma chave publica
    def encapsula(self, h):
        
        # coins <-$ {0,1}^256
        coins = self.pke.randomBitString(256)
        # (r,m) <- Sample_rm(coins)
        (r,m) = self.pke.Sample_rm(self.pke.randomBitString(11200))
        # c <- Encrypt(h,(r,m))
        c = self.pke.cifra(h, (r,m))
        # k <- H1(r,m)
        k = self.Hash1(r,m)
        
        return (c,k)
     
    # Funcao usada para desencapsular uma chave, a partir do seu "encapsulamento" e da chave privada
    def desencapsula(self, sk, c):
        
        # (r,m,fail) <- Decrypt((f,fp,hq),c)
        (r,m,fail) = self.pke.decifra((sk[0], sk[1], sk[2]), c)
        # k1 <- H1(r,m)
        k1 = self.Hash1(r,m)
        # k2 <- H2(s, c)
        k2 = self.Hash2(sk[3],c)
        # if fail = 0 return k1 else return k2
        if fail == 0:
            return k1
        else:
            return k2
        
        

#### Testagem da classe definida acima:

In [11]:
# Parametros do NTRU (ntruhps4096821)
N=821
Q=4096
D=495

# Inicializacao da classe
ntru = NTRU_KEM(N,Q,D)

print("[Teste do encapsulamento e desencapsulamento]")
keys1 = ntru.geraChaves(ntru.pke.randomBitString(2*D))

(c,k) = ntru.encapsula(keys1['pk'])
print("Chave = " + k)

k1 = ntru.desencapsula(keys1['sk'], c)
print("Chave = " + k1)

if k == k1:
    print("A chave desencapsulada é igual à resultante do encapsulamento!!!")
else:
    print("O desencapsulamento falhou!!!")

[Teste do encapsulamento e desencapsulamento]
Chave = 54fcf61bbf13292f02dbce009dde08b8598a0c47f97326e47951baa24f62e2fe
Chave = 54fcf61bbf13292f02dbce009dde08b8598a0c47f97326e47951baa24f62e2fe
A chave desencapsulada é igual à resultante do encapsulamento!!!


### Algumas experiências

In [36]:
n = 256
q = 3329

Qq = PolynomialRing(GF(q), 'x')
y = Qq.gen()
Rq = QuotientRing(Qq, y^n+1)

def poly_to_bytes(poly):
    
    r = []
    for i in range(n//2):
        
        t0 = poly[2*i]
        t1 = poly[2*i+1]
        r.append((t0 >> 0))
        r.append((t0 >> 8) or (t1 << 4))
        r.append((t1 >> 4))
    
    return r


def bytes_to_poly(a):
    
    r = [0]*n
    for i in range(n//2):
        r[2*i]   = ((a[3*i+0] >> 0) or (a[3*i+1] << 8))
        r[2*i+1] = ((a[3*i+1] >> 4) or (a[3*i+2] << 4))
        
    return Rq(r)

pol = Rq.random_element()

pb = poly_to_bytes(pol)

pol1 = bytes_to_poly(pb)

print(pol == pol1)

False
